# Store Sales Analysis (TSA)

Link to Kaggle Notebook :

https://www.kaggle.com/kashishrastogi/store-sales-analysis-time-serie

A must read before starting this competition :

File Descriptions and Data Field Information

train.csv
The training data, comprising time series of features store_nbr, family, and onpromotion as well as the target sales.
store_nbr identifies the store at which the products are sold.
family identifies the type of product sold.
sales gives the total sales for a product family at a particular store at a given date. Fractional values are possible since products can be sold in fractional units (1.5 kg of cheese, for instance, as opposed to 1 bag of chips).
onpromotion gives the total number of items in a product family that were being promoted at a store at a given date.

test.csv
The test data, having the same features as the training data. You will predict the target sales for the dates in this file.
The dates in the test data are for the 15 days after the last date in the training data.

sample_submission.csv
A sample submission file in the correct format.

stores.csv
Store metadata, including city, state, type, and cluster.
cluster is a grouping of similar stores.

oil.csv
Daily oil price. Includes values during both the train and test data timeframes. (Ecuador is an oil-dependent country and it's economical health is highly vulnerable to shocks in oil prices.)

holidays_events.csv

Holidays and Events, with metadata

NOTE: Pay special attention to the transferred column. A holiday that is transferred officially falls on that calendar day, but was moved to another date by the government. A transferred day is more like a normal day than a holiday. To find the day that it was actually celebrated, look for the corresponding row where type is Transfer. For example, the holiday Independencia de Guayaquil was transferred from 2012-10-09 to 2012-10-12, which means it was celebrated on 2012-10-12. Days that are type Bridge are extra days that are added to a holiday (e.g., to extend the break across a long weekend). These are frequently made up by the type Work Day which is a day not normally scheduled for work (e.g., Saturday) that is meant to payback the Bridge.
Additional holidays are days added a regular calendar holiday, for example, as typically happens around Christmas (making Christmas Eve a holiday).


Additional Notes
Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month. Supermarket sales could be affected by this.
A magnitude 7.8 earthquake struck Ecuador on April 16, 2016. People rallied in relief efforts donating water and other first need products which greatly affected supermarket sales for several weeks after the earthquake.

# Importing Modules

In [1]:
import pandas as pd
import numpy as np
import calendar

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.offline as offline
import plotly.graph_objs as go
offline.init_notebook_mode(connected = True)

In [2]:
fig=go.Figure()
fig.add_trace(go.Scatter(
    x=[0, 1, 2, 3],
    y=[1.6, 1.6, 1.6, 1.6],
    mode="text", 
    text=["<span style='font-size:33px'><b>54</b></span>", 
          "<span style='font-size:33px'><b>33</b></span>",
          "<span style='font-size:33px'><b>16</b></span>",
          "<span style='font-size:33px'><b>56</b></span>"],
    textposition="bottom center"
))
fig.add_trace(go.Scatter(
    x=[0, 1, 2, 3],
    y=[1.1, 1.1, 1.1, 1.1],
    mode="text", 
    text=["Stores", "Products", "States", "Months"],
    textposition="bottom center"
))
fig.add_hline(y=2.2, line_width=5, line_color='gray')
fig.add_hline(y=0.3, line_width=3, line_color='gray')
fig.update_yaxes(visible=False)
fig.update_xaxes(visible=False)
fig.update_layout(showlegend=False, height=300, width=700, 
                  title='Store Sales Summary', title_x=0.5, title_y=0.9,
                  xaxis_range=[-0.5,3.6], yaxis_range=[-0.2,2.2],
                  plot_bgcolor='#fafafa', paper_bgcolor='#fafafa',
                  font=dict(size=23, color='#323232'),
                  title_font=dict(size=35, color='#222'),
                  margin=dict(t=90,l=70,b=0,r=70), 
    )

In [4]:
df_holi = pd.read_csv('holidays_events.csv')
df_oil = pd.read_csv('oil.csv')
df_stores = pd.read_csv('stores.csv')
df_trans = pd.read_csv('transactions.csv')

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [5]:
print(df_train.shape)
df_train.head()

(3000888, 6)


,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0.0
1,1,2013-01-01,1,BABY CARE,0.0,0.0
2,2,2013-01-01,1,BEAUTY,0.0,0.0
3,3,2013-01-01,1,BEVERAGES,0.0,0.0
4,4,2013-01-01,1,BOOKS,0.0,0.0


In [6]:
# copying of train data and merging other data
df_train1 = df_train.merge(df_holi, on = 'date', how='left')
df_train1 = df_train1.merge(df_oil, on = 'date', how='left')
df_train1 = df_train1.merge(df_stores, on = 'store_nbr', how='left')
df_train1 = df_train1.merge(df_trans, on = ['date', 'store_nbr'], how='left')
df_train1 = df_train1.rename(columns = {"type_x" : "holiday_type", "type_y" : "store_type"})

df_train1['date'] = pd.to_datetime(df_train1['date'])
df_train1['year'] = df_train1['date'].dt.year
df_train1['month'] = df_train1['date'].dt.month
df_train1['week'] = df_train1['date'].dt.isocalendar().week
df_train1['quarter'] = df_train1['date'].dt.quarter
df_train1['day_of_week'] = df_train1['date'].dt.day_name()

In [9]:
print(df_train1.shape)
df_train1.head()

(3054348, 22)


,id,date,store_nbr,family,sales,onpromotion,holiday_type,locale,locale_name,description,...,city,state,store_type,cluster,transactions,year,month,week,quarter,day_of_week
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,Holiday,National,Ecuador,Primer dia del ano,...,Quito,Pichincha,D,13,NaN,2013,1,1,1,Tuesday
1,1,2013-01-01,1,BABY CARE,0.0,0.0,Holiday,National,Ecuador,Primer dia del ano,...,Quito,Pichincha,D,13,NaN,2013,1,1,1,Tuesday
2,2,2013-01-01,1,BEAUTY,0.0,0.0,Holiday,National,Ecuador,Primer dia del ano,...,Quito,Pichincha,D,13,NaN,2013,1,1,1,Tuesday
3,3,2013-01-01,1,BEVERAGES,0.0,0.0,Holiday,National,Ecuador,Primer dia del ano,...,Quito,Pichincha,D,13,NaN,2013,1,1,1,Tuesday
4,4,2013-01-01,1,BOOKS,0.0,0.0,Holiday,National,Ecuador,Primer dia del ano,...,Quito,Pichincha,D,13,NaN,2013,1,1,1,Tuesday


# AVERAGE SALES ANALYSIS 

sales average by store type

In [10]:
df_train1['store_type'].value_counts()

D    1018116
C     848430
A     509058
B     452496
E     226248
Name: store_type, dtype: int64

In [11]:
df_st_sa = df_train1.groupby('store_type').agg({"sales" : "mean"}).reset_index().sort_values(by='sales', ascending=False)
df_st_sa

,store_type,sales
0,A,708.378165
3,D,352.084510
1,B,328.275233
4,E,270.285490
2,C,197.790647


In [17]:
fig = px.bar(df_st_sa, y='sales', x='store_type',color='sales', barmode='group', height=300,width=300)
fig

sales average by family type

In [12]:
df_train1['family'].value_counts()

DAIRY                         92556
CELEBRATION                   92556
CLEANING                      92556
BABY CARE                     92556
BOOKS                         92556
GROCERY I                     92556
HARDWARE                      92556
MAGAZINES                     92556
HOME AND KITCHEN I            92556
DELI                          92556
BEVERAGES                     92556
HOME CARE                     92556
LADIESWEAR                    92556
SCHOOL AND OFFICE SUPPLIES    92556
PREPARED FOODS                92556
MEATS                         92556
BREAD/BAKERY                  92556
LINGERIE                      92556
HOME AND KITCHEN II           92556
LAWN AND GARDEN               92556
PERSONAL CARE                 92556
HOME APPLIANCES               92556
PRODUCE                       92556
LIQUOR,WINE,BEER              92556
FROZEN FOODS                  92556
SEAFOOD                       92556
GROCERY II                    92556
BEAUTY                      

In [13]:
df_fa_sa = df_train1.groupby('family').agg({"sales" : "mean"}).reset_index().sort_values(by='sales', ascending=False)[:10]
df_fa_sa

,family,sales
12,GROCERY I,3790.432797
3,BEVERAGES,2394.912701
30,PRODUCE,1355.373698
7,CLEANING,1074.171518
8,DAIRY,711.175991
5,BREAD/BAKERY,464.150612
28,POULTRY,351.078816
24,MEATS,341.965905
25,PERSONAL CARE,271.192381
9,DELI,265.629746


In [20]:
fig = px.bar(df_fa_sa, y='sales', x='family',color='sales', barmode='group', height=400,width=400)
fig.update_layout(xaxis_tickangle=-45)
fig

sales average by cluster type

In [22]:
df_train1['cluster'].value_counts()

3     395934
6     339372
10    339372
15    282810
14    226248
13    226248
1     169686
11    169686
4     169686
8     169686
9     113124
2     113124
7     113124
12     56562
5      56562
16     56562
17     56562
Name: cluster, dtype: int64

In [23]:
df_cl_sa = df_train1.groupby('cluster').agg({"sales" : "mean"}).reset_index() 
df_cl_sa

,cluster,sales
0,1,327.022808
1,2,261.025731
2,3,194.926534
3,4,297.537877
4,5,1120.118405
5,6,344.315436
6,7,139.772222
7,8,648.976066
8,9,275.725879
9,10,257.006585


In [24]:
fig = px.bar(df_cl_sa, y='sales', x='cluster',color='sales', barmode='group', height=500,width=500)
fig

Single Chart 

In [35]:
# chart color
df_fa_sa['color'] = '#d5a932'
df_fa_sa['color'][2:] = '#ffda8f'
df_cl_sa['color'] = '#ffda8f'
irises_colors = ['rgb(33, 75, 99)', 'rgb(79, 129, 102)', 'rgb(151, 179, 100)',
                 'rgb(175, 49, 35)', 'rgb(36, 73, 147)']
# chart
fig = make_subplots(rows=2, cols=2, 
                    specs=[[{"type": "bar"}, {"type": "pie"}],
                           [{"colspan": 2}, None]],
                    column_widths=[0.7, 0.3], vertical_spacing=0, horizontal_spacing=0.02,
                    subplot_titles=("Top 10 Highest Product Sales", "Highest Sales in Stores", "Clusters Vs Sales"))

fig.add_trace(go.Bar(x=df_fa_sa['sales'], y=df_fa_sa['family'], marker=dict(color= df_fa_sa['color']),
                     name='Family', orientation='h'), 
                     row=1, col=1)
fig.add_trace(go.Pie(values=df_st_sa['sales'], labels=df_st_sa['store_type'], name='Store type',
                     marker_colors = irises_colors, hole=0.7,
                     hoverinfo='label+percent+value', textinfo='label'), 
                    row=1, col=2)
fig.add_trace(go.Bar(x=df_cl_sa['cluster'], y=df_cl_sa['sales'], 
                     marker=dict(color= df_cl_sa['color']), name='Cluster'), 
                     row=2, col=1)

# styling
fig.update_yaxes(showgrid=False, ticksuffix=' ', categoryorder='total ascending', row=1, col=1)
fig.update_xaxes(visible=False, row=1, col=1)
fig.update_xaxes(tickmode = 'array', tickvals=df_cl_sa.cluster, ticktext=[i for i in range(1,17)], row=2, col=1)
fig.update_yaxes(visible=False, row=2, col=1)
fig.update_layout(height=500, bargap=0.2,
                  margin=dict(b=0,r=20,l=20), xaxis=dict(tickmode='linear'),
                  title_text="Average Sales Analysis",
                  template="plotly_white",
                  title_font=dict(size=29, color='#8a8d93', family="Lato, sans-serif"),
                  font=dict(color='#8a8d93'), 
                  hoverlabel=dict(bgcolor="#f2f2f2", font_size=13, font_family="Lato, sans-serif"),
                  showlegend=False)
fig.show()

Highest sales are made by the products like grocery and beverages.
Store A has the highest sales which is 38%.

# AVERAGE SALES ANALYSIS BY YEAR - MONTH

In [38]:
# data 
df_2013 = df_train1[df_train1['year']==2013][['month','sales']]
df_2013 = df_2013.groupby('month').agg({"sales" : "mean"}).reset_index().rename(columns={'sales':'sa13'})
df_2013

,month,sa13
0,1,186.952405
1,2,193.581846
2,3,206.880581
3,4,205.639071
4,5,210.184563
5,6,215.691343
6,7,203.983455
7,8,212.479434
8,9,220.593588
9,10,213.164266


In [40]:
fig = px.bar(df_2013, y='sa13', x='month',color='sa13', barmode='group', height=500,width=500)
fig

In [41]:
df_2014 = df_train1[df_train1['year']==2014][['month','sales']]
df_2014 = df_2014.groupby('month').agg({"sales" : "mean"}).reset_index().rename(columns={'sales':'sa14'})
df_2014

,month,sa14
0,1,342.341709
1,2,241.268892
2,3,368.661236
3,4,240.577087
4,5,242.203129
5,6,244.634652
6,7,350.830102
7,8,251.351805
8,9,374.530792
9,10,369.213666


In [42]:
fig = px.bar(df_2014, y='sa14', x='month',color='sa14', barmode='group', height=500,width=500)
fig

In [43]:
df_2015 = df_train1[df_train1['year']==2015][['month','sales']]
df_2015 = df_2015.groupby('month').agg({"sales" : "mean"}).reset_index().rename(columns={'sales':'sa15'})
df_2015

,month,sa15
0,1,269.666595
1,2,275.420792
2,3,282.368624
3,4,279.743138
4,5,320.958116
5,6,397.249619
6,7,403.030170
7,8,415.692304
8,9,434.734053
9,10,432.248428


In [44]:
fig = px.bar(df_2015, y='sa15', x='month',color='sa15', barmode='group', height=500,width=500)
fig

In [45]:
df_2016 = df_train1[df_train1['year']==2016][['month','sales']]
df_2016 = df_2016.groupby('month').agg({"sales" : "mean"}).reset_index().rename(columns={'sales':'sa16'})
df_2016

,month,sa16
0,1,434.050268
1,2,424.695398
2,3,418.735398
3,4,488.108774
4,5,457.671398
5,6,419.644575
6,7,432.562218
7,8,406.437390
8,9,419.331240
9,10,435.002169


In [46]:
fig = px.bar(df_2016, y='sa16', x='month',color='sa16', barmode='group', height=500,width=500)
fig

In [47]:
df_2017 = df_train1[df_train1['year']==2017][['month','sales']]
df_2017 = df_2017.groupby('month').agg({"sales" : "mean"}).reset_index()
df_2017_no = pd.DataFrame({'month': [9,10,11,12], 'sales':[0,0,0,0]})
df_2017 = df_2017.append(df_2017_no).rename(columns={'sales':'sa17'})
df_2017

,month,sa17
0,1,476.596791
1,2,465.971468
2,3,483.400632
3,4,482.172948
4,5,487.162797
5,6,488.707278
6,7,489.909880
7,8,465.144891
0,9,0.000000
1,10,0.000000


In [49]:
fig = px.bar(df_2017, y='sa17', x='month',color='sa17', barmode='group', height=500,width=500)
fig

Collective Monthly Sales Growth on Yearly Basis

In [50]:
df_year = df_2013.merge(df_2014,on='month').merge(df_2015,on='month').merge(df_2016,on='month').merge(df_2017,on='month')

df_year

,month,sa13,sa14,sa15,sa16,sa17
0,1,186.952405,342.341709,269.666595,434.050268,476.596791
1,2,193.581846,241.268892,275.420792,424.695398,465.971468
2,3,206.880581,368.661236,282.368624,418.735398,483.400632
3,4,205.639071,240.577087,279.743138,488.108774,482.172948
4,5,210.184563,242.203129,320.958116,457.671398,487.162797
5,6,215.691343,244.634652,397.249619,419.644575,488.707278
6,7,203.983455,350.830102,403.030170,432.562218,489.909880
7,8,212.479434,251.351805,415.692304,406.437390,465.144891
8,9,220.593588,374.530792,434.734053,419.331240,0.000000
9,10,213.164266,369.213666,432.248428,435.002169,0.000000


In [52]:
# top levels
top_labels = ['2013', '2014', '2015', '2016', '2017']

colors = ['rgba(38, 24, 74, 0.8)', 'rgba(71, 58, 131, 0.8)',
          'rgba(122, 120, 168, 0.8)', 'rgba(164, 163, 204, 0.85)',
          'rgba(190, 192, 213, 1)']

# X axis value 
df_year = df_year[['sa13','sa14','sa15','sa16','sa17']].replace(np.nan,0)
x_data = df_year.values

# y axis value (Month)
df_2013['month'] =['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
y_data = df_2013['month'].tolist()

x_data[0]

array([186.95240464, 342.34170923, 269.66659529, 434.05026765,
       476.59679113])

In [54]:
y_data

['Jan',
 'Feb',
 'Mar',
 'Apr',
 'May',
 'Jun',
 'Jul',
 'Aug',
 'Sep',
 'Oct',
 'Nov',
 'Dec']

In [57]:
fig = go.Figure()
for i in range(0, len(x_data[0])):
    for xd, yd in zip(x_data, y_data):
        fig.add_trace(go.Bar(
            x=[xd[i]], y=[yd],
            orientation='h',
            marker=dict(
                color=colors[i],
                line=dict(color='rgb(248, 248, 249)', width=1)
            )
        ))

fig.update_layout(title='Avg Sales for each Year',
    xaxis=dict(showgrid=False, 
               zeroline=False, domain=[0.15, 1]),
    yaxis=dict(showgrid=False, showline=False,
               showticklabels=False, zeroline=False),
    barmode='stack', 
    template="plotly_white",
    margin=dict(l=0, r=50, t=100, b=10),
    showlegend=False, 
)

annotations = []
for yd, xd in zip(y_data, x_data):
    # labeling the y-axis
    annotations.append(dict(xref='paper', yref='y',
                            x=0.14, y=yd,
                            xanchor='right',
                            text=str(yd),
                            font=dict(family='Arial', size=14,
                                      color='rgb(67, 67, 67)'),
                            showarrow=False, align='right'))
    # labeling the first Likert scale (on the top)
    if yd == y_data[-1]:
        annotations.append(dict(xref='x', yref='paper',
                                x=xd[0] / 2, y=1.1,
                                text=top_labels[0],
                                font=dict(family='Arial', size=14,
                                          color='rgb(67, 67, 67)'),
                          showarrow=False))
    space = xd[0]
    for i in range(1, len(xd)):
            # labeling the Likert scale
            if yd == y_data[-1]:
                annotations.append(dict(xref='x', yref='paper',
                                        x=space + (xd[i]/2), y=1.1,
                                        text=top_labels[i],
                                        font=dict(family='Arial', size=14,
                                                  color='rgb(67, 67, 67)'),
                                        showarrow=False))
            space += xd[i]
fig.update_layout(
    annotations=annotations)
fig.show()

Highest sales are made in December month and then decreases in January.
Sales are increasing gradually from 2013 to 2017.

# AVERAGE SALES ANALYSIS TIME SERIES

In [63]:
df_m_sa = df_train1.groupby('month').agg({"sales" : "mean"}).reset_index()
df_m_sa['sales'] = round(df_m_sa['sales'],2)
df_m_sa['month_text'] = df_m_sa['month'].apply(lambda x: calendar.month_abbr[x])
df_m_sa['text'] = df_m_sa['month_text'] + ' - ' + df_m_sa['sales'].astype(str) 

df_w_sa = df_train1.groupby('week').agg({"sales" : "mean"}).reset_index() 
df_q_sa = df_train1.groupby('quarter').agg({"sales" : "mean"}).reset_index() 

#df_q_sa
#df_w_sa
df_m_sa

,month,sales,month_text,text
0,1,341.92,Jan,Jan - 341.92
1,2,320.93,Feb,Feb - 320.93
2,3,352.01,Mar,Mar - 352.01
3,4,341.17,Apr,Apr - 341.17
4,5,345.65,May,May - 345.65
5,6,352.51,Jun,Jun - 352.51
6,7,376.41,Jul,Jul - 376.41
7,8,336.99,Aug,Aug - 336.99
8,9,362.30,Sep,Sep - 362.3
9,10,362.41,Oct,Oct - 362.41


In [70]:
# chart color
df_m_sa['color'] = '#d5a932'
df_m_sa['color'][:-1] = '#ffda8f'
df_w_sa['color'] = '#ffda8f'
irises_colors = ['rgb(33, 75, 99)', 'rgb(79, 129, 102)', 'rgb(151, 179, 100)',
                 'rgb(175, 49, 35)', 'rgb(36, 73, 147)']

# chart
fig = make_subplots(rows=2, cols=2, vertical_spacing=0.08,
                    row_heights=[0.7, 0.3], 
                    specs=[[{"type": "bar"}, {"type": "pie"}],
                           [{"colspan": 2}, None]],
                    column_widths=[0.7, 0.3],
                    subplot_titles=("Month wise Avg Sales Analysis", "Quarter wise Avg Sales Analysis", 
                                    "Week wise Avg Sales Analysis"))

fig.add_trace(go.Bar(x=df_m_sa['sales'], y=df_m_sa['month'], marker=dict(color= df_m_sa['color']),
                     text=df_m_sa['text'],textposition='auto',
                     name='Month', orientation='h'), 
                     row=1, col=1)
fig.add_trace(go.Pie(values=df_q_sa['sales'], labels=df_q_sa['quarter'], name='Quarter',
                     marker_colors= irises_colors, hole=0.7,
                     hoverinfo='label+percent+value', textinfo='label+percent'), 
                     row=1, col=2)
fig.add_trace(go.Scatter(x=df_w_sa['week'], y=df_w_sa['sales'], mode='lines+markers', fill='tozeroy', fillcolor='#ffda8f',
                     marker=dict(color= '#496595'), name='Week'), 
                     row=2, col=1)

# styling
fig.update_yaxes(visible=False, row=1, col=1)
fig.update_xaxes(visible=False, row=1, col=1)
fig.update_xaxes(tickmode = 'array', tickvals=df_w_sa.week, ticktext=[i for i in range(1,53)], 
                 row=2, col=1)
fig.update_yaxes(visible=False, row=2, col=1)
fig.update_layout(height=750, bargap=0.15,
                  margin=dict(b=0,r=20,l=20), 
                  title_text="Average Sales Analysis",
                  template="plotly_white",
                  title_font=dict(size=25, color='#8a8d93', family="Lato, sans-serif"),
                  font=dict(color='#8a8d93'),
                  hoverlabel=dict(bgcolor="#eadbc8", font_size=13, font_family="Lato, sans-serif"),
                  showlegend=False)
fig.show()

#   AVERAGE SALES IN DAY OF WEEK

In [71]:
df_dw_sa = df_train1.groupby('day_of_week').agg({"sales" : "mean"}).reset_index()
df_dw_sa.sales = round(df_dw_sa.sales, 2)

df_dw_sa

,day_of_week,sales
0,Friday,326.73
1,Monday,348.16
2,Saturday,434.79
3,Sunday,464.74
4,Thursday,286.57
5,Tuesday,319.92
6,Wednesday,330.77


In [85]:
# chart
fig = px.bar(df_dw_sa, y='day_of_week', x='sales', title='Avg Sales vs Day of Week',
             color_discrete_sequence=['#ffda8f'], text='sales',
             category_orders=dict(day_of_week=["Monday","Tuesday","Wednesday","Thursday", "Friday","Saturday","Sunday"]))
fig.update_yaxes(showgrid=False, ticksuffix=' ', showline=False)
fig.update_xaxes(visible=False)
fig.update_layout(margin=dict(t=60, b=0, l=0, r=0), height=350,
                  hovermode="y unified", 
                  yaxis_title=" ", template='plotly_white',
                  title_font=dict(size=25, color='#8a8d93', family="Lato, sans-serif"),
                  font=dict(color='#000000'),
                  hoverlabel=dict(bgcolor="#d68b00", font_size=13, font_family="Lato, sans-serif"))

As we saw in the above chart there is an upward trend in sales over the time. Although there are ups and downs at every point in time, generally we can observe that the trend increases. Also we can notice how the ups and downs seem to be a bit regular, it means we might be observing a seasonal pattern here too. Let’s take a closer look by observing some year’s data:
Highest sales are made on Sunday.
December month has the highest sales

In [86]:
#THEEND